## Semi-manual updates

In [274]:
import wled_common_client, importlib    
importlib.reload(wled_common_client)
from wled_common_client import Wled, Wleds

In [275]:
from omegaconf import DictConfig, OmegaConf, ListConfig

In [276]:
w = Wled("192.168.0.122")
w.cache_fs()
w.name = w.cfg["id"]["name"]
wleds = Wleds.from_one_node(w)

In [277]:
for wled in wleds:
    wled.cache_fs()
    # wled.dump_fs("config_dump_names/{name}")

In [278]:
for wled in wleds:
    wled.dump_omegaconf()

In [258]:
for wled in wleds:
    wled.presets = OmegaConf.to_container(OmegaConf.load("default_presets.yaml"))
    wled.upload_presets()

In [279]:
for wled in wleds:
    oc_dir = wled.get_fs_dump_dir("omegaconf_source/{name}")
    cfg_yaml = f"{oc_dir}/cfg.yaml"
    ww = Wled.from_omegaconf(additional_confs=[cfg_yaml])
    ww.ip = wled.ip
    ww.upload_cfg()

In [ ]:
for wled in wleds:
    oc_dir = self.get_fs_dump_dir(wled_common_client.OMEGACONF_DUMP_DIR)
    cfg_yaml = f"{oc_dir}/cfg.yaml"
    presets_yaml = f"{oc_dir}/presets.yaml"
    wled.presets = OmegaConf.load(presets_yaml)
    wled.cfg = OmegaConf.load(presets_yaml)

In [ ]:
yaml.load()

## Tidying up presets

In [158]:
import json
from ruamel.yaml import YAML
yaml = YAML()

In [119]:
with open("config_dump/WLED-Dima-Office/presets.json") as js:
    p = yaml.load(js)

In [120]:
# yaml.compact_seq_seq = True
# yaml.compact_seq_map = False
yaml.width = 10**7
yaml.top_level_colon_align = True


In [121]:
p.fa.set_block_style()
for k in p.keys():
    p[k].fa.set_block_style()

In [122]:
with open("default_presets.yaml", "w") as y:
    yaml.dump(p, y)
